# Data Wrangler for Neural Net Stock Price Problem

This notebook contains scripts that retrieve stock price data, computes a data frame with the necessary transformations and writes the dataframe to a `.csv` file.
This `.csv` file is the tidy data set that we'll use in the Stock Price Neural Network problem.

## Getting started

The organization Quandl supplies financial and economic data in several easy-to-consume formats for free. We'll get our stock price data from Quandl. To do so, first, you'll need the `quandl` python package. Get this via the following:
```pip install quandl```.

In [1]:
import pandas as pd
import quandl
import numpy as np

Next, you'll need a quandl API key, which you can obtain from [Quandl](https://docs.quandl.com/docs#section-authentication). Once you have your key, put it in a YAML file under Quandl with key 'apikey' or just replace the 'YourQuandlAPIKey' string with yours and comment out the yaml code.

In [2]:
#get my quandl api key
import yaml

#comment out the next three lines if just supply your apikey
with open('./databases.yaml', 'r') as f:
        dbparams = yaml.load(f)
apikey = dbparams['Quandl']['apikey']

#comment out three lines above if you use this
#apikey = 'YourQuandlAPIKey'

quandl.ApiConfig.api_key = apikey

## Data from Quandl

In [3]:
# 2018 data for Cisco
data = quandl.get("WIKI/CSCO", start_date = '2018-01-01')

In [4]:
data.tail(10)


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-14,45.34,45.7587,45.0900,45.28,20923845.0,0.0,1.0,45.34,45.7587,45.0900,45.28,20923845.0
2018-03-15,45.30,45.7350,45.1200,45.33,23338222.0,0.0,1.0,45.30,45.7350,45.1200,45.33,23338222.0
2018-03-16,45.33,45.6000,44.9700,45.01,52355707.0,0.0,1.0,45.33,45.6000,44.9700,45.01,52355707.0
2018-03-19,44.59,44.8200,43.9000,44.27,24524286.0,0.0,1.0,44.59,44.8200,43.9000,44.27,24524286.0
2018-03-20,44.49,44.6400,44.1800,44.37,22385001.0,0.0,1.0,44.49,44.6400,44.1800,44.37,22385001.0
2018-03-21,44.24,44.9000,44.1331,44.31,20616375.0,0.0,1.0,44.24,44.9000,44.1331,44.31,20616375.0
2018-03-22,43.76,44.0200,43.0200,43.07,29374734.0,0.0,1.0,43.76,44.0200,43.0200,43.07,29374734.0
2018-03-23,43.71,43.8400,42.4200,42.42,30674112.0,0.0,1.0,43.71,43.8400,42.4200,42.42,30674112.0
2018-03-26,43.25,44.1600,42.8300,44.06,28454954.0,0.0,1.0,43.25,44.1600,42.8300,44.06,28454954.0


In [5]:
#list of dow companies from https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average 
dow_companies = ['MMM','AXP', 'AAPL','BA','CAT',
                 'CVX','CSCO','KO','DWDP','XOM',
                 'GS','HD','IBM','INTC','JNJ','JPM',
                 'MCD','MRK','MSFT','NKE','PFE',
                 'PG','TRV','UNH','UTX','VZ','V',
                 'WMT','WBA','DIS']

In [6]:
len(dow_companies)

30

## Calcuate Moving Averages and Buy and Sell Signals

In [7]:
#get the data for the first 3 dow companies:

cols = ['Open','High','Low','Close','Volume']
company = 'CSCO' #Cisco Systems Inc
dow_data = quandl.get('WIKI/'+company, start_date = '2018-01-01')[cols]

In [8]:
dow_data.keys()


Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [9]:
dow_data.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2018-03-21,44.24,44.90,44.1331,44.31,20616375.0
2018-03-22,43.76,44.02,43.0200,43.07,29374734.0
2018-03-23,43.71,43.84,42.4200,42.42,30674112.0
2018-03-26,43.25,44.16,42.8300,44.06,28454954.0
2018-03-27,44.49,44.52,42.2400,42.68,30088447.0


In [10]:
# calculate the moving averages just on the closing price
#9-day trailing moving average
MA9 =  dow_data[['Close']].rolling(window= 9, center=False).mean().rename(columns={'Close':'MA9'})
MA18 = dow_data[['Close']].rolling(window=18, center=False).mean().rename(columns={'Close':'MA18'})

In [11]:
# paste the moving averages onto the right side of the dataframe
dow_data = dow_data.merge(MA9,  how='left', left_index=True, right_index=True)
dow_data = dow_data.merge(MA18, how='left', left_index=True, right_index=True)



In [12]:
dow_data

,Open,High,Low,Close,Volume,MA9,MA18
Date,,,,,,,
2018-01-02,38.670,38.9500,38.4300,38.86,19972431.0,NaN,NaN
2018-01-03,38.720,39.2850,38.5300,39.17,29066090.0,NaN,NaN
2018-01-04,39.050,39.5400,38.9300,38.99,20606344.0,NaN,NaN
2018-01-05,39.550,39.8800,39.3650,39.53,24369510.0,NaN,NaN
2018-01-08,39.520,39.9600,39.3500,39.94,16511704.0,NaN,NaN
2018-01-09,39.790,39.9600,39.5400,39.69,21339760.0,NaN,NaN
2018-01-10,39.650,40.2400,39.6300,39.91,19110146.0,NaN,NaN
2018-01-11,40.140,40.2100,39.7500,40.10,20178596.0,NaN,NaN
2018-01-12,40.220,40.9300,40.0500,40.87,22962700.0,39.673333,NaN


In [13]:
# is the fast moving average (MA9) greater then the slow moving average (MA18)?
dow_data['f_gtr_s'] = (dow_data.MA9 > dow_data.MA18)*1 # need these as ints
dow_data['f_gtr_s'] = np.where(np.logical_or(np.isnan(dow_data.MA9), np.isnan(dow_data.MA18)), np.NaN, dow_data.f_gtr_s)

In [14]:
dow_data

,Open,High,Low,Close,Volume,MA9,MA18,f_gtr_s
Date,,,,,,,,
2018-01-02,38.670,38.9500,38.4300,38.86,19972431.0,NaN,NaN,NaN
2018-01-03,38.720,39.2850,38.5300,39.17,29066090.0,NaN,NaN,NaN
2018-01-04,39.050,39.5400,38.9300,38.99,20606344.0,NaN,NaN,NaN
2018-01-05,39.550,39.8800,39.3650,39.53,24369510.0,NaN,NaN,NaN
2018-01-08,39.520,39.9600,39.3500,39.94,16511704.0,NaN,NaN,NaN
2018-01-09,39.790,39.9600,39.5400,39.69,21339760.0,NaN,NaN,NaN
2018-01-10,39.650,40.2400,39.6300,39.91,19110146.0,NaN,NaN,NaN
2018-01-11,40.140,40.2100,39.7500,40.10,20178596.0,NaN,NaN,NaN
2018-01-12,40.220,40.9300,40.0500,40.87,22962700.0,39.673333,NaN,NaN


In [15]:
# find the transition points by comparing consecutive days
# first, add column that is the lagged MA comparison (lagged by one day)
dow_data['f_gtr_s_lag1'] = dow_data.f_gtr_s.shift()
#use boolean arithmetic to see the change points
dow_data['Crossover'] = dow_data.f_gtr_s - dow_data.f_gtr_s_lag1

In the table below, a Crossover value of 1 indicates a row (a date) on which 9-day moving average transitioned from being less than the 18-day moving average to being greater than the 18-day moving average.
Conversely, a crossover value of -1 indicates a date on which the 9-day moving average transitioned from being greater than the 18-day moving average to being less than the 18-day moving averages.
***A Crossover value of 1 is a BUY signal; a Crossover value of -1 is a SELL signal***

In [16]:
dow_data

,Open,High,Low,Close,Volume,MA9,MA18,f_gtr_s,f_gtr_s_lag1,Crossover
Date,,,,,,,,,,
2018-01-02,38.670,38.9500,38.4300,38.86,19972431.0,NaN,NaN,NaN,NaN,NaN
2018-01-03,38.720,39.2850,38.5300,39.17,29066090.0,NaN,NaN,NaN,NaN,NaN
2018-01-04,39.050,39.5400,38.9300,38.99,20606344.0,NaN,NaN,NaN,NaN,NaN
2018-01-05,39.550,39.8800,39.3650,39.53,24369510.0,NaN,NaN,NaN,NaN,NaN
2018-01-08,39.520,39.9600,39.3500,39.94,16511704.0,NaN,NaN,NaN,NaN,NaN
2018-01-09,39.790,39.9600,39.5400,39.69,21339760.0,NaN,NaN,NaN,NaN,NaN
2018-01-10,39.650,40.2400,39.6300,39.91,19110146.0,NaN,NaN,NaN,NaN,NaN
2018-01-11,40.140,40.2100,39.7500,40.10,20178596.0,NaN,NaN,NaN,NaN,NaN
2018-01-12,40.220,40.9300,40.0500,40.87,22962700.0,39.673333,NaN,NaN,NaN,NaN


In [17]:
#ditch the rows with NAs due to the trailing moving averages
dow_data.dropna(axis=0,inplace=True)
dow_data

,Open,High,Low,Close,Volume,MA9,MA18,f_gtr_s,f_gtr_s_lag1,Crossover
Date,,,,,,,,,,
2018-01-29,42.300,42.9800,42.3000,42.85,23053104.0,41.892222,40.876111,1.0,1.0,0.0
2018-01-30,42.685,42.8600,41.9700,42.25,25671649.0,42.008889,41.047222,1.0,1.0,0.0
2018-01-31,41.980,42.0100,41.3500,41.54,34571282.0,42.035556,41.188889,1.0,1.0,0.0
2018-02-01,41.090,42.1100,40.6700,41.70,26001606.0,42.081111,41.309444,1.0,1.0,0.0
2018-02-02,41.500,41.9500,40.8700,40.93,25920378.0,42.000000,41.364444,1.0,1.0,0.0
2018-02-05,40.870,41.4050,39.0800,39.09,39491216.0,41.665556,41.331111,1.0,1.0,0.0
2018-02-06,38.330,40.3050,37.3500,40.17,52940550.0,41.443333,41.345556,1.0,1.0,0.0
2018-02-07,40.310,41.2060,40.0300,40.34,31841942.0,41.270000,41.358889,0.0,1.0,-1.0
2018-02-08,40.440,40.7600,38.7200,38.73,39300118.0,40.844444,41.240000,0.0,0.0,0.0


## Get the HLOCV for the Purchase Date and 4 Previous Days

In [18]:
# first purchase date
bd = dow_data.query('Crossover==1').index[0]

In [19]:
# row in the data frame of the purchase date
indx = dow_data.index.get_loc(bd)

In [20]:
indx

15

In [21]:
cols = ['Open','High','Low','Close','Volume']

In [22]:
# these will eventually be our predictors (25 for each purchase)
dow_data.iloc[indx-4:indx+1][cols]

,Open,High,Low,Close,Volume
Date,,,,,
2018-02-13,40.500,41.31,40.22,41.23,29220084.0
2018-02-14,41.040,42.26,40.99,42.09,39880261.0
2018-02-15,45.065,45.13,43.26,44.08,71490591.0
2018-02-16,43.885,45.09,43.79,44.33,37915915.0
2018-02-20,44.000,44.69,43.81,44.06,26258211.0


In [23]:
# turn the above into a one-row data frame.
#First, we need some column names: XX_n will stand for variable XX at n days before purchase date
#e.g., Column 'Open_4' represents open price 4 days before purchase date
colnames = [c+'_'+str(p) for p in [4,3,2,1]  for c in cols ] + cols

In [24]:
str(colnames)

"['Open_4', 'High_4', 'Low_4', 'Close_4', 'Volume_4', 'Open_3', 'High_3', 'Low_3', 'Close_3', 'Volume_3', 'Open_2', 'High_2', 'Low_2', 'Close_2', 'Volume_2', 'Open_1', 'High_1', 'Low_1', 'Close_1', 'Volume_1', 'Open', 'High', 'Low', 'Close', 'Volume']"

In [25]:
pd.DataFrame(dow_data.iloc[indx-4:indx+1][cols].values.reshape(1,-1),columns = colnames, index=[bd])

,Open_4,High_4,Low_4,Close_4,Volume_4,Open_3,High_3,Low_3,Close_3,Volume_3,...,Open_1,High_1,Low_1,Close_1,Volume_1,Open,High,Low,Close,Volume
2018-02-20,40.5,41.31,40.22,41.23,29220084.0,41.04,42.26,40.99,42.09,39880261.0,...,43.885,45.09,43.79,44.33,37915915.0,44.0,44.69,43.81,44.06,26258211.0


In [26]:
#function to get the HLOCV info for 4 days before the buy date(bd)
#returns a wide data frame
def getPredictors(bd, history, n=5):
    
    # row in the data frame of the buy date
    indx = history.index.get_loc(bd)
    
    # column names of interest in the history dataframe
    cols = ['Open','High','Low','Close','Volume']
    # column names for the df to be returned:
    colnames = [c+'_'+str(abs(p)) for p in range(-1*(n-1),0,1)  for c in cols ] + cols
    
    #make a row vector out of the data to be returned:
    rv = history.iloc[(indx-(n-1)):indx+1][cols].values.reshape(1,-1)
    
    #handle case where there's insufficient history - just return df with all NaNs.
    if rv.size != len(colnames):
        rv = np.full((1,len(colnames)), np.nan)
    
    return(pd.DataFrame(rv, columns = colnames, index = [bd]))

In [27]:
getPredictors(bd, dow_data)

,Open_4,High_4,Low_4,Close_4,Volume_4,Open_3,High_3,Low_3,Close_3,Volume_3,...,Open_1,High_1,Low_1,Close_1,Volume_1,Open,High,Low,Close,Volume
2018-02-20,40.5,41.31,40.22,41.23,29220084.0,41.04,42.26,40.99,42.09,39880261.0,...,43.885,45.09,43.79,44.33,37915915.0,44.0,44.69,43.81,44.06,26258211.0


In [28]:
xx=dow_data.index[0]
xx

Timestamp('2018-01-29 00:00:00')

In [29]:
getPredictors(xx,dow_data)

,Open_4,High_4,Low_4,Close_4,Volume_4,Open_3,High_3,Low_3,Close_3,Volume_3,...,Open_1,High_1,Low_1,Close_1,Volume_1,Open,High,Low,Close,Volume
2018-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sale Results

Stocks are purchased on a 'buy' signal. Once purchased, stocks are sold on the earliest of three dates:

1. 'Sell' signal (crossover value is -1 indicating that the fast moving average transitioned from being higher than the slow moving average to being below the slow moving average
1. maximum number of trading days to hold the stock has been exceeded
1. value of stock decreased below some threshold amount

In [78]:
# function to compute the result of a buy.
# given a buy date and the stock history, loss threshold, max trading days hold time,
# returns the date of sale, reason for sale and the price at sale date

def saleResult(bd,history, loss_thresh = 0.2, maxhold = 20):
    
    reasons = ['SellSig', 'MaxHold', 'LossThresh']
    future_date = pd.to_datetime('2200-12-31') # way after we're all gone
    
    # not interested in the history before the buy data
    df = history.loc[bd:] # this makes the record for the buy date the 0'th record, very handy!
    
    # first sell signal after the buy date; Crossover value of -1 is a sell signal
    ss = future_date
    sell_signals = df.index[df.Crossover == -1]
    if sell_signals.size > 0:
        ss = sell_signals[0]
    
    # max trading days
    md = future_date
    if df.index.size >= maxhold:
        md = df.index[maxhold]
        
    # loss_threshold
    lt = future_date
    maxloss = 1-loss_thresh
    buy_price = df.Close[0]
    min_price = buy_price*maxloss
    lossdates = df.index[df.Close <= min_price]
    if lossdates.size > 0:
        lt = lossdates[0]
    
    # figure out which is the earliest date
    exit_dates = np.array([ss, md, lt]) # need to be in same order as reasons list
    exit_date_i = exit_dates.argmin()
    
    exit_date = exit_dates[exit_date_i]
    reason = reasons[exit_date_i]
    
    # get the exit price
    rn = df.index.get_loc(exit_date)
    exit_price = df.Close[rn]
    
    return (exit_date, reason, exit_price)
    
    

In [68]:
saleResult(bd, dow_data)

(Timestamp('2018-03-20 00:00:00'), 'MaxHold', 44.369999999999997)

In [65]:
bd

Timestamp('2018-02-20 00:00:00')

In [64]:
dow_data

,Open,High,Low,Close,Volume,MA9,MA18,f_gtr_s,f_gtr_s_lag1,Crossover
Date,,,,,,,,,,
2018-01-29,42.300,42.9800,42.3000,42.85,23053104.0,41.892222,40.876111,1.0,1.0,0.0
2018-01-30,42.685,42.8600,41.9700,42.25,25671649.0,42.008889,41.047222,1.0,1.0,0.0
2018-01-31,41.980,42.0100,41.3500,41.54,34571282.0,42.035556,41.188889,1.0,1.0,0.0
2018-02-01,41.090,42.1100,40.6700,41.70,26001606.0,42.081111,41.309444,1.0,1.0,0.0
2018-02-02,41.500,41.9500,40.8700,40.93,25920378.0,42.000000,41.364444,1.0,1.0,0.0
2018-02-05,40.870,41.4050,39.0800,39.09,39491216.0,41.665556,41.331111,1.0,1.0,0.0
2018-02-06,38.330,40.3050,37.3500,40.17,52940550.0,41.443333,41.345556,1.0,1.0,0.0
2018-02-07,40.310,41.2060,40.0300,40.34,31841942.0,41.270000,41.358889,0.0,1.0,-1.0
2018-02-08,40.440,40.7600,38.7200,38.73,39300118.0,40.844444,41.240000,0.0,0.0,0.0


In [79]:
saleResult(bd, dow_data,loss_thresh=0.01)

(Timestamp('2018-02-21 00:00:00'), 'LossThresh', 43.310000000000002)

In [74]:
1 - 0.05

0.95